In [1]:
import pandas as pd

import json

In [2]:
import pandas as pd
import json

def load_dataset(file_name):
    df = []
    with open(file_name, encoding="utf-8") as f:
        for i in f:
            i = json.loads(i)
            label = i['tags']
            label = label[0]
            
            if label == 'multi':
                continue
            
            df += [{'postText': i['postText'],
                    'targetParagraphs': i['targetParagraphs'],
                    'targetTitle': i['targetTitle'],
                    'targetDescription': i['targetDescription'],
                    'targetKeywords': i['targetKeywords'],
                    'spoiler': i['spoiler'],
                     'labels': (label == 'phrase' or label == 'phrases')}]

    return pd.DataFrame(df)

In [3]:
train_dataset = load_dataset('/content/train.jsonl')
validation_dataset = load_dataset('/content/validation.jsonl')

In [4]:
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel
import sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 

In [5]:
import re
import pandas as pd
import json
import argparse
from os.path import exists
from glob import glob
from os.path import isdir
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import warnings
import nltk
warnings.simplefilter(action='ignore')
import spacy
from nltk.corpus import wordnet
from nltk import pos_tag

In [6]:
train_dataset

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,labels
0,"[Wes Welker Wanted Dinner With Tom Brady, But ...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,[NASA sets date for full recovery of ozone hole],[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,[This is what makes employees happy -- and it'...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,[The perfect way to cook rice so that it's per...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,[What happens if your new AirPods get lost or ...,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,[If You See A Purple Butterfly Sticker At The ...,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,[Has Facebook's video explosion completely sha...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,[Cop Is Eating At A Chili's When Teen Hands Hi...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,[You need to see this Twitter account that pre...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
nlp = spacy.load("en_core_web_sm")
if not nlp.vocab:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [9]:
def alter_nouns_with_synonyms(text):
    doc = nlp(text)
    result = []
    for token in doc:
        if token.pos_ == 'NOUN':
            synonyms = []
            for syn in wordnet.synsets(token.text):
                for lemma in syn.lemmas():
                    synonyms.append(lemma.name())
            if synonyms:
                changed = False
                for syn in synonyms:
                    if syn != token.text:
                        changed = True
                        result.append(syn)
                        break
                if not changed:
                    result.append(token.text)
            else:
                result.append(token.text)
        else:
            result.append(token.text)
    return ' '.join(result)




In [10]:
def get_data(dataset_to_change):

  df_result = []
  for i in range(len(dataset_to_change)):
      post_text = dataset_to_change.iloc[i]['postText'][0]
      altered_text = alter_nouns_with_synonyms(post_text)
      temp_res = {'postText': [altered_text],
                  'targetParagraphs': dataset_to_change.iloc[i]['targetParagraphs'],
                  'targetTitle': dataset_to_change.iloc[i]['targetTitle'],
                  'targetDescription': dataset_to_change.iloc[i]['targetDescription'],
                  'targetKeywords': dataset_to_change.iloc[i]['targetKeywords'],
                  'spoiler': dataset_to_change.iloc[i]['spoiler'],
                  'labels': dataset_to_change.iloc[i]['labels']}
      df_result.append(temp_res)

  altered_dataframe = pd.DataFrame(df_result, columns=['postText', 'targetParagraphs',
                                                     'targetTitle', 'targetDescription',
                                                     'targetKeywords', 'spoiler',
                                                     'labels'])
  
  return altered_dataframe

In [11]:
train_new = get_data(train_dataset)
validation_new = get_data(validation_dataset)



In [12]:
train_new

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,labels
0,"[Wes Welker Wanted Dinner With Tom Brady , But...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,[NASA sets day_of_the_month for full convalesc...,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,[This is what makes employee happy -- and it '...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,[The perfect manner to cook Rice so that it 's...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,[What happens if your new AirPods get lost or ...,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,[If You See A Purple Butterfly Sticker At The ...,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,[Has Facebook 's picture detonation completely...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,[bull Is Eating At A Chili 's When Teen Hands ...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,[You need to see this Twitter history that pre...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [13]:
# datasets = [train_dataset, train_new]
# result = pd.concat(datasets)

In [14]:
# result

In [15]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
import re 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [16]:
import nltk 
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
def lemmit(row):
  lemm_list = []
  for text in row['postText']:
    doc = nlp(text)
    for word in doc:
      if not word.is_punct and not word.is_stop:
        lemm_list.append(word.lemma_)
    break
  return ' '.join(lemm_list)


In [18]:
train_new['postText'] = train_new.apply(lemmit, axis=1)
validation_new['postText'] = validation_new.apply(lemmit, axis=1)

In [19]:
len(validation_new)

657

In [20]:
import pandas as pd
import json

def preprocess_dataframe(df):

    processed_df = []

    for index, row in df.iterrows():
        tweet = row['postText']
        article_title = row['targetTitle']
        article = ' '.join(row['targetParagraphs'])
        label = row['labels']

        processed_df.append({'text': tweet + " - " + article_title + article,
                             'labels': label})

    return pd.DataFrame(processed_df)

In [21]:
train_dataset = preprocess_dataframe(train_new)
validation_dataset = preprocess_dataframe(validation_new)

In [22]:
train_dataset

,text,labels
0,Wes Welker want Dinner Tom Brady Patriots QB B...,False
1,NASA set day_of_the_month convalescence ozone ...,True
2,make employee happy payroll_check - Intellectu...,True
3,perfect manner cook Rice perfectly fluffy stic...,True
4,happen new AirPods lose steal Apple - Here's w...,False
...,...,...
2636,Purple Butterfly Sticker Hospital mean - If Yo...,False
2637,Facebook picture detonation completely shake a...,False
2638,bull eat Chili Teen hand Folded Napkin walk aw...,False
2639,need Twitter history predict Beyoncé gestation...,True


In [23]:
import torch
torch.cuda.empty_cache()

final_model = None

config = {
    'overwrite_output_dir': True,
    'num_train_epochs': 5,
    'fp16': False,
    'train_batch_size': 8,
    'gradient_accumulation_steps': 4,
    'evaluate_during_training': True,
    'max_seq_length': 512,
    'learning_rate': 4e-05,
    'early_stopping_consider_epochs': True,
    'early_stopping_delta': 0.01,
    'early_stopping_metric': 'acc',
    'early_stopping_metric_minimize': False,
    'early_stopping_patience': 3,
    'evaluate_during_training_steps': 331,
    'output_dir': 'outputs/',
    'warmup_ratio': 0.06,
    'save_steps': 2000,
    'manual_seed': 12345,
    }

model = ClassificationModel("roberta", "roberta-base", args = config)
model.train_model(train_dataset, eval_df=train_dataset, acc=sklearn.metrics.accuracy_score)
result = model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/2641 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/2641 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/2641 [00:00<?, ?it/s]

  0%|          | 0/2641 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

In [25]:
result

({'mcc': 0.4859770985104088,
  'tp': 263,
  'tn': 225,
  'fp': 97,
  'fn': 72,
  'auroc': 0.8024844720496894,
  'auprc': 0.8098396455225441,
  'acc': 0.7427701674277016,
  'eval_loss': 0.7752663432564362},
 array([[ 1.5141629 , -1.3220197 ],
        [-1.81159055,  2.19899774],
        [ 1.93800843, -1.8422488 ],
        ...,
        [-1.44940531,  1.82327044],
        [ 1.66821027, -1.58910811],
        [-2.25960779,  2.57888126]]),
 [{'guid': 11, 'text_a': 'bird fart - Do birds fart?Have you ever wondered, "Do birds fart?" No? Well maybe now we’ve sparked your curiosity. They don’t fart—detectably. There’s no reason to believe that birds are somehow exempt from passing gas, but it doesn’t seem like anyone has ever truly observed—or more importantly, measured—a bird fart. But between ornithologists, veterinarians, and amateur bird watchers, how is it possible no one has ever seen the feathered creatures flatulate? As Monterey Bay Aquarium veterinarian Mike Murray said in a 2009 article